In [1]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
## manually inspecting the vehicle data reveals there are three work sheets ##
## The work sheet we care about is the 'County' worksheet as a df. ##
countyvehicle_df = pd.read_excel('vehicle_data/california_vehicle.xlsx',sheet_name="County")

In [3]:
## We've isolated the worksheet, now let's take a look at the shape ##
countyvehicle_df.shape


(33542, 7)

In [4]:
## We see there are 33542 rows and 7 columns, now let's see what that looks like ##
countyvehicle_df.head()

,Data Year,County,Dashboard Fuel Type Group,Fuel Type,Make,Model,Number of Vehicles
0,2010,Alameda,Battery Electric (BEV),Battery Electric (BEV),Ford,Ranger,3
1,2010,Alameda,Battery Electric (BEV),Battery Electric (BEV),Tesla,Roadster,17
2,2010,Alameda,Diesel,Diesel,NaN,NaN,10939
3,2010,Alameda,Gasoline,Flex Fuel,NaN,NaN,10974
4,2010,Alameda,Gasoline,Gasoline,NaN,NaN,840577


In [5]:
## We're not interested in the Make or Model of the car, so let's drop those values ##
countyvehicle_df = countyvehicle_df.drop(columns=['Make','Model'])

In [6]:
countyvehicle_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33542 entries, 0 to 33541
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Data Year                  33542 non-null  int64 
 1   County                     33542 non-null  object
 2   Dashboard Fuel Type Group  33542 non-null  object
 3   Fuel Type                  33542 non-null  object
 4   Number of Vehicles         33542 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 1.3+ MB


In [7]:
countyvehicle_df['County'].unique()

array(['Alameda', 'Alpine', 'Amador', 'Butte', 'Calaveras', 'Colusa',
       'Contra Costa', 'Del Norte', 'El Dorado', 'Fresno', 'Glenn',
       'Humboldt', 'Imperial', 'Inyo', 'Kern', 'Kings', 'Lake', 'Lassen',
       'Los Angeles', 'Madera', 'Marin', 'Mariposa', 'Mendocino',
       'Merced', 'Modoc', 'Mono', 'Monterey', 'Napa', 'Nevada', 'Orange',
       'Out Of State', 'Placer', 'Plumas', 'Riverside', 'Sacramento',
       'San Benito', 'San Bernardino', 'San Diego', 'San Francisco',
       'San Joaquin', 'San Luis Obispo', 'San Mateo', 'Santa Barbara',
       'Santa Clara', 'Santa Cruz', 'Shasta', 'Sierra', 'Siskiyou',
       'Solano', 'Sonoma', 'Stanislaus', 'Sutter', 'Tehama', 'Trinity',
       'Tulare', 'Tuolumne', 'Ventura', 'Yolo', 'Yuba', 'Out of State'],
      dtype=object)

In [8]:
## There are vehicles present that are from out of state. Let's see what years have vehicles from out of state ##
for year in countyvehicle_df['Data Year'].unique():
    out_of_state_2023 = countyvehicle_df[(countyvehicle_df['Data Year'] == year) & 
                                         (countyvehicle_df['County'] == 'Out of State')]
    
    total_out_of_state_2023 = out_of_state_2023['Number of Vehicles'].sum()
    if total_out_of_state_2023 != 0:
        print(f"The total number of vehicles for 'Out of State' in {year} is: {total_out_of_state_2023}")

The total number of vehicles for 'Out of State' in 2021 is: 403531
The total number of vehicles for 'Out of State' in 2022 is: 409644


In [9]:
## Looks like only 2021 and 2022 have vehicles from out of state present ##
## Since out of state is only present for two years, we'll drop those values and note our assumption that they have a marginal impact ##
calcountyvehicle_df = countyvehicle_df[~countyvehicle_df['County'].isin(['Out of State'])]

In [10]:
## Next, let's take a look at the fuel types that are used. We're interested in impact of EVs and non-fossil fuel based cars ##
## There are two columns that cover the fuel type, 'Dashboard Fuel Type Group' and 'Fuel Type' that are fully populated (length 33542) ##
## Let's take a look at the difference between the two ##
fuel_types_1 = calcountyvehicle_df['Dashboard Fuel Type Group'].unique()
fuel_types_2 = calcountyvehicle_df['Fuel Type'].unique()

fuel_types_df = pd.DataFrame({
    'Fuel Type': pd.Series(fuel_types_2),
    'Dashboard Fuel Type Group': pd.Series(fuel_types_1)
})

fuel_types_df


,Fuel Type,Dashboard Fuel Type Group
0,Battery Electric (BEV),Battery Electric (BEV)
1,Diesel,Diesel
2,Flex Fuel,Gasoline
3,Gasoline,Gasoline Hybrid
4,Gasoline Hybrid,Other
5,Natural Gas,Fuel Cell (FCEV)
6,Propane,Plug-in Hybrid (PHEV)
7,Fuel Cell (FCEV),NaN
8,Plug-in Hybrid (PHEV),NaN


In [11]:
## It appears Dashboard Fuel Type Group goes ahead and clusters them together ## 
## We're going to make our own the assumption and group together vehicles based on fuel type ##
## Diesel, Flex Fuel, Gasoline, Gasoline Hybrid, Natural Gas, and Propane will be treated as Fossil Fuels ##
## Battery Electric, Fuel Cell, and Plug-in Hybrid will be treated as EVs ##
fossil_fuels = ['Diesel', 'Flex Fuel', 'Gasoline', 'Gasoline Hybrid', 'Natural Gas', 'Propane']
evs = ['Battery Electric (BEV)', 'Fuel Cell (FCEV)', 'Plug-in Hybrid (PHEV)']
def categorize_fuel(fuel_type):
    if fuel_type in fossil_fuels:
        return 'Fossil Fuel'
    elif fuel_type in evs:
        return 'EV'
    else:
        return 'Other'
calcountyvehicle_df.loc[:, 'Fuel Category'] = calcountyvehicle_df['Fuel Type'].apply(categorize_fuel)
# Print the DataFrame to check the new column
calcountyvehicle_df.head()

/var/folders/zc/cqjcs2293rz1ymjq5gj5xxb00000gn/T/ipykernel_56576/232989404.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calcountyvehicle_df.loc[:, 'Fuel Category'] = calcountyvehicle_df['Fuel Type'].apply(categorize_fuel)


,Data Year,County,Dashboard Fuel Type Group,Fuel Type,Number of Vehicles,Fuel Category
0,2010,Alameda,Battery Electric (BEV),Battery Electric (BEV),3,EV
1,2010,Alameda,Battery Electric (BEV),Battery Electric (BEV),17,EV
2,2010,Alameda,Diesel,Diesel,10939,Fossil Fuel
3,2010,Alameda,Gasoline,Flex Fuel,10974,Fossil Fuel
4,2010,Alameda,Gasoline,Gasoline,840577,Fossil Fuel


In [12]:
fueltype_per_countyyear_totals_df = calcountyvehicle_df.groupby(['Data Year','County','Fuel Category'])['Number of Vehicles'].sum().reset_index()
fueltype_per_countyyear_totals_df

,Data Year,County,Fuel Category,Number of Vehicles
0,2010,Alameda,EV,20
1,2010,Alameda,Fossil Fuel,885402
2,2010,Alpine,Fossil Fuel,1041
3,2010,Amador,EV,1
4,2010,Amador,Fossil Fuel,30480
...,...,...,...,...
1594,2023,Ventura,Fossil Fuel,651851
1595,2023,Yolo,EV,6757
1596,2023,Yolo,Fossil Fuel,152560
1597,2023,Yuba,EV,961


In [13]:
# to csv
fueltype_per_countyyear_totals_df.to_csv('/Users/clarissaclark/SIADS_Milestone_1/fueltype_per_countyyear_totals_df.csv', index=False)


In [14]:
for year in fueltype_per_countyyear_totals_df['Data Year'].unique():
    for county in fueltype_per_countyyear_totals_df['County'].unique():
        # Get the subset of the dataframe for this year and county
        subset = fueltype_per_countyyear_totals_df[(fueltype_per_countyyear_totals_df['Data Year'] == year) & 
                                                   (fueltype_per_countyyear_totals_df['County'] == county)]
        
        # Check if 'EV' exists in the 'Fuel Category' for this year and county
        if 'EV' not in subset['Fuel Category'].values:
            # If not, create a new row with 0 vehicles for 'EV'
            new_row_ev = pd.DataFrame({'Data Year': [year], 'County': [county], 'Fuel Category': ['EV'], 'Number of Vehicles': [0]})
            fueltype_per_countyyear_totals_df = pd.concat([fueltype_per_countyyear_totals_df, new_row_ev], ignore_index=True)

        # Check if 'Fossil Fuel' exists in the 'Fuel Category' for this year and county
        if 'Fossil Fuel' not in subset['Fuel Category'].values:
            # If not, create a new row with 0 vehicles for 'Fossil Fuel'
            new_row_fossil = pd.DataFrame({'Data Year': [year], 'County': [county], 'Fuel Category': ['Fossil Fuel'], 'Number of Vehicles': [0]})
            fueltype_per_countyyear_totals_df = pd.concat([fueltype_per_countyyear_totals_df, new_row_fossil], ignore_index=True)

# Optionally, sort the dataframe by 'Data Year', 'County', and 'Fuel Category' for better readability
fueltype_per_countyyear_totals_df = fueltype_per_countyyear_totals_df.sort_values(by=['Data Year', 'County', 'Fuel Category']).reset_index(drop=True)

fueltype_per_countyyear_totals_df.head()

,Data Year,County,Fuel Category,Number of Vehicles
0,2010,Alameda,EV,20
1,2010,Alameda,Fossil Fuel,885402
2,2010,Alpine,EV,0
3,2010,Alpine,Fossil Fuel,1041
4,2010,Amador,EV,1


In [15]:
import pandas as pd
import os
from urllib.request import urlretrieve
pd.options.mode.chained_assignment = None  # default='warn'

def categorize_fuel(fuel_type):
    """
    Categorize fuel types into 'Fossil Fuel', 'EV', or 'Other'.
    
    Args:
        fuel_type (str): The fuel type to categorize.
    
    Returns:
        str: The category of the fuel type.
    """
    fossil_fuels = ['Diesel', 'Flex Fuel', 'Gasoline', 'Gasoline Hybrid', 'Natural Gas', 'Propane']
    evs = ['Battery Electric (BEV)', 'Fuel Cell (FCEV)', 'Plug-in Hybrid (PHEV)']
    
    if fuel_type in fossil_fuels:
        return 'Fossil Fuel'
    elif fuel_type in evs:
        return 'EV'
    else:
        return 'Other'
def download_file(url, local_filename):
    """
    Download a file from a URL to a local filename.
    
    Args:
        url (str): The URL of the file to download.
        local_filename (str): The local file path to save the downloaded file.
    """
    try:
        urlretrieve(url, local_filename)
        print(f"File downloaded successfully and saved as {local_filename}.")
    except Exception as e:
        raise Exception(f"An error occurred while downloading the file: {e}")
    
def fueltype_per_countyyear_totals_df(excel_file):
    """
    Generate a DataFrame summarizing the number of vehicles by year, county, and fuel category.
    
    Args:
        excel_file (str): Path to the Excel file containing vehicle data.
    
    Returns:
        pd.DataFrame: A DataFrame with totals of vehicles by year, county, and fuel category.
    
    Raises:
        FileNotFoundError: If the specified Excel file is not found.
        ValueError: If the sheet 'County' is not found in the Excel file or if there are issues with the data format.
        """
    excel_file = os.path.join(os.getcwd(), excel_file)

    if not os.path.exists(excel_file):
        print(f"File {excel_file} not found locally. Attempting to download...")
        URL = 'https://www.energy.ca.gov/filebrowser/download/6311?fid=6311#block-symsoft-page-title'
        download_file(URL, excel_file)
    
    # Pull County Sheet from excel file ##
    try:
        # Read the Excel file
        countyvehicle_df = pd.read_excel(excel_file, sheet_name="County")
    except FileNotFoundError:
        raise FileNotFoundError(f"The file {excel_file} was not found.")
    except ValueError as e:
        raise ValueError("An error occurred while reading the Excel file. Ensure the file contains a sheet named 'County'.") from e
    except Exception as e:
        raise Exception("An unexpected error occurred while reading the Excel file.") from e
    
    ## Drop Irrelevent Columns ##
    try:
        countyvehicle_df = countyvehicle_df.drop(columns=['Make', 'Model'])
    except KeyError as e:
        raise KeyError("The specified columns to drop do not exist in the DataFrame.") from e
    
    
    ## Remove unwanted County Information ##
    cal_countyvehicle_df = countyvehicle_df[~countyvehicle_df['County'].isin(['Out of State'])]
    
    ## Simply fuel types and categories ##
    cal_countyvehicle_df.loc[:, 'Fuel Category'] = cal_countyvehicle_df['Fuel Type'].apply(categorize_fuel)
    
    ## Create final data frame based on year, county, fuel category, and number of vehicles ##
    cartype_df = cal_countyvehicle_df.groupby(['Data Year', 'County', 'Fuel Category'])['Number of Vehicles'].sum().reset_index()  
    for year in cartype_df['Data Year'].unique():
        for county in cartype_df['County'].unique():
            # Get the subset of the dataframe for this year and county
            subset = cartype_df[(cartype_df['Data Year'] == year) & 
                                                       (cartype_df['County'] == county)]

            # Check if 'EV' exists in the 'Fuel Category' for this year and county
            if 'EV' not in subset['Fuel Category'].values:
                # If not, create a new row with 0 vehicles for 'EV'
                new_row_ev = pd.DataFrame({'Data Year': [year], 'County': [county], 'Fuel Category': ['EV'], 'Number of Vehicles': [0]})
                cartype_df = pd.concat([cartype_df, new_row_ev], ignore_index=True)

            # Check if 'Fossil Fuel' exists in the 'Fuel Category' for this year and county
            if 'Fossil Fuel' not in subset['Fuel Category'].values:
                # If not, create a new row with 0 vehicles for 'Fossil Fuel'
                new_row_fossil = pd.DataFrame({'Data Year': [year], 'County': [county], 'Fuel Category': ['Fossil Fuel'], 'Number of Vehicles': [0]})
                cartype_df = pd.concat([cartype_df, new_row_fossil], ignore_index=True)

    # Optionally, sort the dataframe by 'Data Year', 'County', and 'Fuel Category' for better readability
    cartype_df = cartype_df.sort_values(by=['Data Year', 'County', 'Fuel Category']).reset_index(drop=True)
    output_path = 'fueltype_per_countyyear_totals.csv'
    cartype_df.to_csv(output_path, index=False)
    return cartype_df

fueltype_per_countyyear_df =  fueltype_per_countyyear_totals_df('vehicle_data/california_vehicle.xlsx') 
fueltype_per_countyyear_df.head

<bound method NDFrame.head of       Data Year   County Fuel Category  Number of Vehicles
0          2010  Alameda            EV                  20
1          2010  Alameda   Fossil Fuel              885402
2          2010   Alpine            EV                   0
3          2010   Alpine   Fossil Fuel                1041
4          2010   Amador            EV                   1
...         ...      ...           ...                 ...
1647       2023  Ventura   Fossil Fuel              651851
1648       2023     Yolo            EV                6757
1649       2023     Yolo   Fossil Fuel              152560
1650       2023     Yuba            EV                 961
1651       2023     Yuba   Fossil Fuel               57567

[1652 rows x 4 columns]>

In [16]:
# # Now that our Primary Dataset is cleaned and maniputed, let's export it to a csv so that other people can use it 
# fueltype_per_countyyear_df.to_csv('./SIADS_Milestone_1/vehicle_df', index = False)

In [18]:
# for year in cartype_df['Data Year'].unique():
#     for county in cartype_df['County'].unique():
#         # Get the subset of the dataframe for this year and county
#         subset = cartype_df[(cartype_df['Data Year'] == year) & 
#                                                    (cartype_df['County'] == county)]
#         
#         # Check if 'EV' exists in the 'Fuel Category' for this year and county
#         if 'EV' not in subset['Fuel Category'].values:
#             # If not, create a new row with 0 vehicles for 'EV'
#             new_row_ev = pd.DataFrame({'Data Year': [year], 'County': [county], 'Fuel Category': ['EV'], 'Number of Vehicles': [0]})
#             cartype_df = pd.concat([cartype_df, new_row_ev], ignore_index=True)
# 
#         # Check if 'Fossil Fuel' exists in the 'Fuel Category' for this year and county
#         if 'Fossil Fuel' not in subset['Fuel Category'].values:
#             # If not, create a new row with 0 vehicles for 'Fossil Fuel'
#             new_row_fossil = pd.DataFrame({'Data Year': [year], 'County': [county], 'Fuel Category': ['Fossil Fuel'], 'Number of Vehicles': [0]})
#             cartype_df = pd.concat([cartype_df, new_row_fossil], ignore_index=True)
# 
# # Optionally, sort the dataframe by 'Data Year', 'County', and 'Fuel Category' for better readability
# cartype_df = cartype_df.sort_values(by=['Data Year', 'County', 'Fuel Category']).reset_index(drop=True)
# 
# cartype_df.head()

In [19]:
## There are two variants of Out of State and 
filtered_df = fueltype_per_county_totals[~fueltype_per_county_totals['County'].isin(['Out of State','Out Of State'])]
filtered_df

NameError: name 'fueltype_per_county_totals' is not defined

In [20]:

filtered_df = calcountyvehicle_df[(calcountyvehicle_df['Data Year'] == 2023) & 
                                  (calcountyvehicle_df['Dashboard Fuel Type Group'] == 'Other')]

# Calculate the sum of the 'Number of Vehicles' for the filtered data
total_vehicles_2023_other = filtered_df['Number of Vehicles'].sum()

# Print the result
print(f"The total number of vehicles in 2023 for 'Other' fuel type is: {total_vehicles_2023_other}")

The total number of vehicles in 2023 for 'Other' fuel type is: 8220


In [21]:
## We are going to assume that 
excluded_types = [fuel_type for fuel_type in fuel_types_2 
                  if fuel_type not in fuel_types_1]
print(excluded_types)

['Flex Fuel', 'Natural Gas', 'Propane']


In [ ]:

    
## 
#county_df['County'].unique()

In [ ]:
# Get unique fuel types from both columns
fuel_types_1 = county_df['Fuel Type'].unique()
fuel_types_2 = calcountyvehicle_df['Dashboard Fuel Type Group'].unique()

# Create a DataFrame to display the values side by side
fuel_types_df = pd.DataFrame({
    'Fuel Type': pd.Series(fuel_types_1),
    'Dashboard Fuel Type Group': pd.Series(fuel_types_2)
})

# Display the DataFrame
fuel_types_df

In [ ]:
cali_county_df = county_df[~county_df['County'].isin(['Out of State','Out Of State'])]
cali_county_df['County'].unique()

In [ ]:
fuel_types = cali_county_df['Fuel Type'].unique()

In [ ]:
statewide_vehicle_totals = county_df.groupby(['Fuel Type', 'Data Year'])['Number of Vehicles'].sum().reset_index()
statewide_vehicle_totals['Fuel Type'].unique()

In [ ]:
total_vehicles_by_year = county_df.groupby('Data Year')['Number of Vehicles'].sum().reset_index()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))

for fuel_type in fuel_types:
    fuel_data = statewide_vehicle_totals[statewide_vehicle_totals['Fuel Type'] == fuel_type]
    plt.plot(fuel_data['Data Year'], fuel_data['Number of Vehicles']/1000000, label=fuel_type)

plt.plot(total_vehicles_by_year['Data Year'], total_vehicles_by_year['Number of Vehicles']/1000000, 
         label='Total Vehicles', color='black', linestyle='--', linewidth=2)

bev_data = statewide_vehicle_totals[statewide_vehicle_totals['Fuel Type'] == 'Battery Electric (BEV)']
gasoline_data = statewide_vehicle_totals[statewide_vehicle_totals['Fuel Type'] == 'Gasoline']

merged_data = pd.merge(bev_data[['Data Year', 'Number of Vehicles']], gasoline_data[['Data Year', 'Number of Vehicles']],
                       on='Data Year', suffixes=('_BEV', '_Gasoline'))

merged_data = pd.merge(merged_data, total_vehicles_by_year[['Data Year', 'Number of Vehicles']], 
                       on='Data Year')
#(merged_data.head())

merged_data['BEV (%)'] = (merged_data['Number of Vehicles_BEV'] / merged_data['Number of Vehicles']) * 100
merged_data['Gasoline (%)'] = (merged_data['Number of Vehicles_Gasoline'] / merged_data['Number of Vehicles']) * 100

#column_labels = ['Year', 'BEV (%)', 'Gasoline (%)']
plt.legend(title='Fuel Type')
#plt.table(cellText=table_data, colLabels=column_labels, loc='right', cellLoc='center', bbox=[0.0, -0.5, 1, 0.3])
ax1 = plt.gca()  # Get current axes instance for the main plot
ax2 = ax1.twinx()  # Create a second axes sharing the same x-axis


ax2.plot(merged_data['Data Year'], merged_data['Gasoline (%)'], color='red', linestyle='--', label='Gasoline (%)')
ax2.set_ylabel('Gasoline Vehicles (%)', color='red')
ax2.tick_params(axis='y', labelcolor='red')
ax2.set_ylim(0, 100)
ax2.set_ylabel('Percentage Total Cars')

plt.title('Total Vehicles by Fuel Type and Year')
plt.xlabel('Year')
ax1.set_ylabel('Number of Vehicles (millions)')
plt.legend()


plt.grid(True)
plt.show()


In [ ]:
merged_data[['Data Year','BEV (%)','Gasoline (%)']].head(14)

In [ ]:
#table_to_be_merged=
## We want to take the data for vehicles by year and include only BEV vehicles for each county ##
def bevpercentagevisual(year):

    bev_data_year = reduced_cali_county_df[(reduced_cali_county_df['Fuel Type'] == 'Battery Electric (BEV)') & 
                                           (reduced_cali_county_df['Data Year'] == year)]
    
    
    # Total vehicles for 2010 and 2023
    total_vehicles_year = reduced_cali_county_df[reduced_cali_county_df['Data Year'] == year].groupby('County')['Number of Vehicles'].sum().reset_index()
    
    # Merge BEV data with total vehicles for each year
    merged_year = pd.merge(bev_data_year.groupby('County')['Number of Vehicles'].sum().reset_index(), total_vehicles_year, on='County', suffixes=('_BEV', '_Total'))
    
    
    # Calculate BEV percentage for 2010 and 2023
    merged_year['BEV (%)'] = (merged_year['Number of Vehicles_BEV'] / merged_year['Number of Vehicles_Total']) * 100
    
    # Step 3: Plot the data side by side
    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(18, 10), sharey=True)
    
    # Plot 2010 data
    axes.barh(merged_year['County'], merged_year['BEV (%)'], color='blue')
    axes.set_xlabel('Percentage of BEVs (%)')
    axes.set_ylabel('County')
    axes.set_title('Percentage of BEVs by County (2010)')
    
    
    
    # Display the plots
    plt.tight_layout()
    plt.show()
bevpercentagevisual(2013)


In [ ]:
import plotly.graph_objects as go

In [ ]:
# Filter data for 2010, 2018, and 2023
years = [2010, 2018,2020, 2023]
filtered_data = reduced_cali_county_df[reduced_cali_county_df['Data Year'].isin(years)]


total_vehicles = filtered_data.groupby(['Data Year', 'County'])['Number of Vehicles'].sum().reset_index()

bev_data = filtered_data[filtered_data['Fuel Type'] == 'Battery Electric (BEV)']
bev_vehicles = bev_data.groupby(['Data Year', 'County'])['Number of Vehicles'].sum().reset_index()

merged_data = pd.merge(bev_vehicles, total_vehicles, on=['Data Year', 'County'], suffixes=('_BEV', '_Total'))

merged_data['BEV (%)'] = (merged_data['Number of Vehicles_BEV'] / merged_data['Number of Vehicles_Total']) * 100


pivot_data = merged_data.pivot(index='County', columns='Data Year', values='BEV (%)').fillna(0)


plt.figure(figsize=(12, 8))


for year, color in zip(years, ['blue', 'orange', 'green','red']):
    plt.barh(pivot_data.index, pivot_data[year], color=color, alpha=0.5, label=f'{year}')


plt.xlabel('Percentage of BEVs (%)')
plt.ylabel('County')
plt.title('Percentage of Battery Electric Vehicles (BEVs) by County for 2010, 2018, and 2023')
plt.legend(title='Year')


plt.tight_layout()
plt.show()